<a href="https://colab.research.google.com/github/toanctran/web_scrapper_selenium_beautifulsoup/blob/master/tuoitre_scraper_Colab_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver-manager

     |████████████████████████████████| 911kB 5.8MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 30 not upgraded.
Need to get 83.2 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 89.0.4389.90-0ubuntu0.18.04.2 [1,127 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 89.0.4389.90-0ubuntu0.18.04.2 [73.6 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from signal import signal, SIGINT
from sys import exit

import pandas as pd

from time import time, sleep
import sys
import random

def handler(signal_received, frame):
    # Handle any cleanup here
    print('SIGINT or CTRL-C detected.Saving data to file')
    df = pd.DataFrame(data=data, columns=data[0].keys())
    # Export and save the DataFrame df to result.csv file
    df.to_csv('result_exit.csv', index=False, encoding='utf_8')
    exit(0)

signal(SIGINT, handler)

print("Running. Press Ctrl+C to exit")


options = webdriver.ChromeOptions()
options.headless = True
options.add_argument("--window-size=1920,1200")
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument("--incognito")

driver = webdriver.Chrome('chromedriver', options=options)

data = []
de_muc_list = ['thoi-su', 'the-gioi' , 'phap-luat', 'kinh-doanh', ] #'cong-nghe', 'xe', 'nhip-song-tre', 'van-hoa','giai-tri', 'giao-duc', 'khoa-hoc', 'suc-khoe' ]
for i in range(0, len(de_muc_list) - 1 ):
    de_muc = de_muc_list[i]
    waiting_page = 0
    n = 1
    found = True
    article_dic = {'category':'', 'article_title' : '', 'article_url':'', 'article':'', 'summary':''}
    
    print(f"Starting scraping the artice in {de_muc}")
    while found:
        article_temp = []
        print(f"Getting page {n} of {de_muc} ...")  
        url = "https://tuoitre.vn/" + de_muc + "/trang-" + str(n) + ".htm"
        driver.get(url)
        try:
            # # Wait until the element with CLASS_NAME = product-item present
            # myElem = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'news-item')))

            # Wait 30s to poll the DOM element when trying to find any element (or elements) not immediately available
            driver.implicitly_wait(30)

            print(f"Page {n} in {de_muc} is ready!")
        except TimeoutException:
            print("Loading took too much time!")
            print("Quitting driver")
            driver.quit()

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        news_list = soup.find_all('div', {'class':'name-news'})

        if news_list == [] and i != len(de_muc_list) - 1:
            found = False
            print("End of {de_muc}. Move to {de_muc_list[i + 1]}")
        elif news_list == [] and i == len(de_muc_list) - 1:
            print("Successfully scraping all the category on TuoiTre")
            print("Quitting driver")
            found = False
            driver.quit()
            break
        
        for news in news_list:
            try:
                article_dic['category'] = de_muc
                try:
                    article_dic['article_title'] = news.a['title']
                except:
                    continue
                article_href = "https://tuoitre.vn" + news.a['href']
                article_dic['article_url'] = article_href

                driver.get(article_href)
                driver.implicitly_wait(30)

                article_page = BeautifulSoup(driver.page_source, 'html.parser')
                article_dic['summary'] = article_page.find('h2', {'class':'sapo'}).text
                
                article_ = article_page.find('div', {"id": "main-detail-body"}).find_all('p')
                if article_page.find('div', {'type':'RelatedOneNews'}) != None:
                    related_ = article_page.find('div', {'type':'RelatedOneNews'}).p.text

                article = ''
                if article_ != []:
                    for item in article_:
                        if related_ != None and item.text == related_:
                            continue
                        else:
                            article = article + item.text + " "
                article_dic['article'] = article
            # Handle the error
            except Exception as e:
                print('We got an error when try to process an article')
                tb = sys.exc_info()[2]
                print(e.with_traceback(tb))
                continue

            if article_dic not in data:
                article_dic_copy = article_dic.copy()
                article_temp.append(article_dic_copy)
                print(f"Scraped {len(article_temp)} articles")

        if article_temp == []:
            print(f'Cannot find new article in page {n} of {de_muc}. Waiting page: {waiting_page} pages')
            waiting_page += 1

        if article_temp == [] and i != len(de_muc_list) - 1 and waiting_page == 5:
            print("End of {de_muc}. Move to {de_muc_list[i + 1]}")
            found = False
            
        elif article_temp == [] and i == len(de_muc_list) - 1  and waiting_page == 5:
            print("Successfully scraping all the category on TuoiTre")
            found = False
            

        if data != [] and all(elem in article_temp for elem in data) and article_temp != [] :
            print("Len of data", len(data))
            print("Len of article_temp", len(article_temp))
            print('Finish scrapping.')
            print('Saving data to result.csv')
            # Create the Pandas DataFrame with the collected data       
            df = pd.DataFrame(data=data, columns=data[0].keys())
            # Export and save the DataFrame df to result.csv file
            df.to_csv('result.csv', index=False, encoding='utf_8')
            print('Successfully saved data to result.csv')
            found = False
            
        print(f"Starting save {len(article_temp)} articles from article_temp to data")
        print(f"Data have {len(data)} articles before saving")
        data = data + article_temp
        print(f"Successfully saving {len(article_temp)} articles from article_temp to data")
        print(f"Data have {len(data)} articles after saving")
        sleep_time = random.randint(5, 10)
        print(f'Scrapper sleep in {sleep_time}')
        sleep(sleep_time)
        print(f'Scrapped page {n} of {de_muc}. Continue to page {n+1} of {de_muc} .')
            
        n += 1
        print('Saving data to result_temp.csv')
        # Create the Pandas DataFrame with the collected data       
        df = pd.DataFrame(data=data, columns=data[0].keys())
        # Export and save the DataFrame df to result.csv file
        df.to_csv('result_temp.csv', index=False, encoding='utf_8')
        print('Successfully saved data to result_temp.csv')
        


# Create the Pandas DataFrame with the collected data       
df = pd.DataFrame(data=data, columns=data[0].keys())
# Export and save the DataFrame df to result.csv file
df.to_csv('result.csv', index=False, encoding='utf_8')

Running. Press Ctrl+C to exit
Starting scraping the artice in thoi-su
Getting page 1 of thoi-su ...
Page 1 in thoi-su is ready!
Scraped 1 articles
We got an error when try to process an article
'NoneType' object has no attribute 'text'
Scraped 2 articles
Scraped 3 articles
Scraped 4 articles
Scraped 5 articles
Scraped 6 articles
Scraped 7 articles
Scraped 8 articles
Scraped 9 articles
Scraped 10 articles
Scraped 11 articles
We got an error when try to process an article
'NoneType' object has no attribute 'text'
Scraped 12 articles
Scraped 13 articles
Scraped 14 articles
Scraped 15 articles
Starting save 15 articles from article_temp to data
Data have 0 articles before saving
Successfully saving 15 articles from article_temp to data
Data have 15 articles after saving
Scrapper sleep in 9
Scrapped page 1 of thoi-su. Continue to page 2 of thoi-su .
Saving data to result_temp.csv
Successfully saved data to result_temp.csv
Getting page 2 of thoi-su ...
Page 2 in thoi-su is ready!
Scraped 1 a